# Getting Started with Qdrant

In the first part of this series, you were introduced to vector databases and why they're essential for building context-aware applications like RAG.

Now that you have a solid understanding of vector databases and Qdrant, it's time to roll up your sleeves and get your hands dirty. In this post, you'll set up your development environment and get started with Qdrant, the vector database that you'll use throughout this series. You'll sign up for a Qdrant cloud account, install the necessary libraries, set up our environment variables, and instantiate a cluster - all the necessary steps to start building something. By the end of this post, you'll have a fully functional Qdrant setup and be ready to start experimenting with vector embeddings and similarity searches. 

### Prepare Your Cloud Enviornment

The easiest way to get started with Qdrant is by using their managed cloud service, Qdrant Cloud. 

You can, of course, use Qdrant locally if you'd like. It will require you to set up Docker on your machine and familiarity with Docker containers. If you're interested in learning how to get setup locally, check out the documentation [here](https://qdrant.tech/documentation/quick-start/). For this series, we're going to work strictly in the cloud. 

To sign up for Qdrant Cloud:


1) Head over to the Qdrant Cloud website.

2) Click on the "Cloud" button in the top right corner.

3) Fill in your details and create your account.

You don't have to set up a paid account, as Qdrant offers a generous free tier that's perfect for development and testing. And, if you end up building a product using billions of vectors Qdrant will smoothly scale with it.

Go and sign up for a [cloud account](https://qdrant.tech).

<div style="text-align: center;">
    <img src="/Users/harpreetsahota/workspace/practical-rag-book/exploring-qdrant/image_assets/qdrant-sign-up.gif" style="width: 75%; height: auto;">
</div>

Now, you'll need to create your your cluser and get your API key. 

To set up a cluster:

• Locate the sidebar on the left hand screen of the console console.

• Click on the "Clusters" menu item.

• Look for the "Create" button on the top right of the screen.

• Type in "Practical_Retrieval_Augmented_Generation" as your cluster name, then press "Create Cluster".

• Once the cluster is created you'll see a red button for "Get API Key", press that button and copy your API key. Be sure to keep it somewhere safe. 

<div style="text-align: center;">
    <img src="/Users/harpreetsahota/workspace/practical-rag-book/exploring-qdrant/image_assets/setup-qdrant-cluser.gif" style="width: 75%; height: auto;">
</div>

You'll also need your Cluster URL, you can get that by:

• Navigating to the Clusters item in the sidebar

• Clicking the right facing carrot ( ͐) on your cluster name

• And you can find it under "Cluster URL". Copy that and keep it safe.

<div style="text-align: center;">
    <img src="/Users/harpreetsahota/workspace/practical-rag-book/exploring-qdrant/image_assets/get-qdrant-cluster-url.gif" style="width: 75%; height: auto;">
</div>

I recommend setting these as environment variables, you can do that by opening up your terminal and running the following (assuming you're on Linux or MacOS):

```shell
export QDRANT_API_KEY=<your-api-key>
export QDRANT_URL=<your-qdrant-cloud-url>
```

I also recommed keeping these in a `.env` file you can easily load them in your notebooks and scripts. You can do that by running the following in your terminal:

```shell
echo "QDRANT_API_KEY=$QDRANT_API_KEY" >> .env && echo "QDRANT_URL=$QDRANT_URL" >> .env
```

Now, set up your development environment.

### Prepare Your Development Enviornment

#### Create a Virtual Enviornment

It's a best practice to create a virtual environment before you start installing libraries and working on a project. So, get that out of the way, then start installing some packages: 

```shell
conda create -n p_rag python==3.10
```

Once that's done, activate your enviornment:

```shell
conda activate p_rag
```

#### Install Dependencies

To interact with Qdrant using the [Python sdk](https://github.com/qdrant/qdrant-client), you'll need to install the Qdrant client library. In addition, at least for right now and at a bare minimum, you'll also need to download:

- [`python-dotenv`](https://github.com/theskumar/python-dotenv)

 - [`Instructor`](https://github.com/jxnl/instructor)
 
 - [`OpenAI`](https://github.com/openai/openai-python)

 - [`sentence-transformers`](https://github.com/openai/openai-python)

 - [`transformers`](https://github.com/openai/openai-python)


No need for `LangChain` or `LlamaIndex`. We're not going to use them in this series. Any other requirements will be installed as needed. Now, open your terminal and run the following commands. Be sure to use the exact same versions as me. 

```shell
pip install python-dotenv==1.0.1 qdrant-client==1.9.0 instructor==1.2.2 openai==1.23.6 transformers==4.40.1 sentence-transformers==2.7.0 
```

Now, open up a Jupyter Notebook. It's time to connect to your cluster programmatically and create your first collection.

Start with the immediately necessary imports:

In [ ]:
import os
from dotenv import load_dotenv

from qdrant_client import QdrantClient

load_dotenv('./.env')

Now, instantiate the client.

In [ ]:
client = QdrantClient(
    url=os.getenv('QDRANT_URL'),
    api_key=os.getenv('QDRANT_API_KEY')
)

Run the following code, and notice that our list of collections is empty. As it should be, because you haven't created one yet.

In [ ]:
client.get_collections()

### WTF Is a Collection?

Before I have you create a collection, it's gonna be useful for me to tell you what they are. A collection in Qdrant holds *points*. Points are the central entity in Qdrant, and they're defined by:

-  **Vector:** Represents the data (like an image, chunk of text, audio segment, video. etc) as an array of floating point numbers. Each vector will have a unique identifier associated with it.

-  **Payload:** Additional information about the data (basically metadata). This is just a JSON object.

Each point within a collection must have a vector of the same dimensionality and be compared using a single similarity metric. This allows you to search points in a collection based on vector similarity. Before creating your collection, there are some design choices you need to make:

 - **Data type:** What kind of data will you store? (e.g., images, text, or both?)

 - **Vector size and distance metric:** Vector size will depend on your chosen embedding model. The distance metric can be one of: dot product, cosine similarity, Euclidean distance, or Manhattan distance.

 - **What to put in the Payload:** Decide what additional metadata you want to store with each vector.

Each collection has its own set of parameters that allow you controls for how its optimized, indexed, and stored. These settings can be changed at any time, and you don't need to worry about them right now. What you do need to worry about right now is how to configure your collection, define the vector dimensionality, and the distance metric. 

Over the next few blogs, you'll work exclusively with text data. For that I'll use OpenAI's `text-embedding-3-large` embedding model, which has a default dimensionality of `3072`. I'll also use cosine similarity as the distance metric.


In [ ]:
from qdrant_client.http import models

collection_config = models.VectorParams(
    size=1536,
    distance=models.Distance.COSINE
)

client.create_collection(
    collection_name="p_rag_series_1",
    vectors_config=collection_config
)

And now you can verify that the collection has been created by running `client.get_collections()` or through the UI as shown below.


<div style="text-align: center;">
    <img src="/Users/harpreetsahota/workspace/practical-rag-book/exploring-qdrant/image_assets/verify-collection-creation.gif" style="width: 75%; height: auto;">
</div>



In [ ]:
client.get_collections()

Now, go ahead and close the client down.

In [17]:
client.close()

### I'm going to jump ahead of myself and briefly discuss two concepts:

1) Named vectors

2) Multitenancy

#### Named Vectors 

A collection can store multiple types of vectors, such as image vectors and text vectors, within the same collection. 

This is achieved using the concept of "named vectors." When creating a collection, you can define multiple named vectors, each with its own unique name, distance metric, and dimensionality. For example, you can have a named vector called "image" for storing image vectors and another named vector called "text" for storing text vectors.

Here's the important thing to note:

 - Within each named vector, all the vectors must have the same dimensionality. So, all the vectors stored under the "image" named vector must have the same number of dimensions, and all the vectors stored under the "text" named vector must have the same number of dimensions.

 - However, the dimensionality of the "image" vectors and the "text" vectors can be different from each other. For instance, the "image" vectors might have a dimensionality of 512, while the "text" vectors might have a dimensionality of 768.

 - Each named vector can also have its own distance metric. So, you can use a different distance metric for comparing image vectors and text vectors.

#### Multitenancy

Imagine you have an apartment building (your Qdrant instance) with many tenants (users). Multitenancy is like having a single, shared laundry room (a collection) instead of each tenant having their own washer and dryer (separate collections). 

Multitenancy in Qdrant means storing data for multiple tenants or users within a single collection.

In most cases, you should use a single collection for all tenants, which is efficient but requires additional configuration. This involves partitioning vectors by adding a tenant-specific field (e.g., group_id) to each vector's payload. To ensure data isolation, you need to filter vectors based on the tenant ID when performing searches or updates. This way, each tenant can only access their own data. Extending the laundry room anology above, it means you're not mixing up peoples undies.

We won't deal with multitenancy in this series, but at least you now know its a thing.

The alternative approach of creating separate collections for each tenant.

This will ensure strict data isolation and flexibility, especially when dealing with a limited number of users. By assigning a dedicated collection to each tenant, their data remains completely segregated, preventing any interference or unauthorized access between tenants.

However, this approach comes with potential drawbacks. 

As the number of tenants increases, managing multiple collections can lead to higher resource overhead and costs compared to using a single collection for all tenants. The administrative effort required to maintain and monitor numerous collections also becomes more complex and time-consuming.

This likely isn't the most efficient or scalable solution when dealing with a large number of tenants. In such cases, using a single collection with proper partitioning and access controls is often more manageable and cost-effective.

### See You In The Next Blog

In the next blog, you'll learn how to go from real world text data to vectors. Then, you'll learn how to populate your Qdrant collection with vectors so you can search over them.
